<a href="https://colab.research.google.com/github/Ersel26/echo_KVAK/blob/main/echo_KVAK_efficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import random
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.optimizers import Adam
from keras.applications import EfficientNetB3

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
DIRECTORY = r"/content/drive/MyDrive/Echo_KVAK/labeled data"
IMG_SIZE = 112 # for EffectiveNetB0
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir(DIRECTORY))

In [68]:
def create_data_ImageDataGenerator(data_gen, data_dir):
  train_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='training'
  ) 
  validation_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='validation')  # set as validation data
  return (train_generator, validation_generator)

In [69]:
data_gen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2)

(train_generator, validation_generator) = create_data_ImageDataGenerator(data_gen, DIRECTORY)

Found 9213 images belonging to 7 classes.
Found 2302 images belonging to 7 classes.


In [70]:
efficient_net = EfficientNetB3(
    weights=None,
    input_shape=(IMG_SIZE,IMG_SIZE,1),
    include_top=False,
    pooling='max'
)

model = tf.keras.models.Sequential()
model.add(efficient_net)
model.add(layers.Dense(units = 120, activation='relu'))
model.add(layers.Dense(units = 120, activation = 'relu'))
model.add(layers.Dense(units = NUM_CLASSES, activation='sigmoid'))
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10782811  
                                                                 
 dense_9 (Dense)             (None, 120)               184440    
                                                                 
 dense_10 (Dense)            (None, 120)               14520     
                                                                 
 dense_11 (Dense)            (None, 7)                 847       
                                                                 
Total params: 10,982,618
Trainable params: 10,895,319
Non-trainable params: 87,299
_________________________________________________________________


In [71]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(
    train_generator,
    epochs = 50,
    steps_per_epoch = 15,
    validation_data = validation_generator,
    validation_steps = 7,
    callbacks = [callback]
)

Epoch 1/50
15/15 [==============================] - 487s 27s/step - loss: 2.0529 - accuracy: 0.2292 - val_loss: 1.8687 - val_accuracy: 0.2902
Epoch 2/50
15/15 [==============================] - 391s 27s/step - loss: 1.8748 - accuracy: 0.2542 - val_loss: 1.8047 - val_accuracy: 0.3214
Epoch 3/50
15/15 [==============================] - 372s 25s/step - loss: 1.8089 - accuracy: 0.2729 - val_loss: 1.7583 - val_accuracy: 0.3348
Epoch 4/50
15/15 [==============================] - 377s 26s/step - loss: 1.8973 - accuracy: 0.2646 - val_loss: 1.7733 - val_accuracy: 0.2768
Epoch 5/50
15/15 [==============================] - 320s 22s/step - loss: 1.8634 - accuracy: 0.2354 - val_loss: 1.7847 - val_accuracy: 0.2991
Epoch 6/50
15/15 [==============================] - 279s 19s/step - loss: 1.8221 - accuracy: 0.2893 - val_loss: 1.7450 - val_accuracy: 0.2946
